# Cvičenie 3: Multilayer Perceptron

Na dnešnom cvičení budete implementovať doprednú fázu viacvrstvového perceptrónu (multilayer perceptron), teda neurónovej siete s viacerými vrstvami. Fungovanie perceptrónu by vám už malo byť jasné, dnes rozšírime štruktúru o niekoľko neurónov, ktoré zoskupujeme do troch vrstiev (vstupná, skrytá a výstupná). Do neurónov takisto pridáme aktivačné funkcie ReLU a sigmoid.

Pred tým než sa spustíte do práce, zopakujte si teoretické znalosti o neurónových sieťach, najmä čo sa týka jednotlivých výpočtov ktoré sa vykonajú v rámci neurónov. Pri diskusii vám môže pomôcť architektúra multilayer perceptrona:

![Štruktúra neurónovej siete](sources/lab03/3.1-mlp-structure.jpeg)

## 1. Prvý pohľad na kód

Stiahnite si [kostru riešenia](sources/lab03/lab3.zip), ktorá obsahuje Python skript s prázdnymi triedami pre implementáciu neurónovej siete. Trieda `Layer` popisuje všeobecné rozhranie jednej vrstvy v neurónovej sieti, ktorá má:
* doprednú fázu (`forward`) - výpočet výstupu na základe vstupu, teda predikcia
* trénovaciu fázu (`backward`) - trénovanie siete, aktualizácia váh.

Vašou úlohou je implementovať vrstvy ` ReLU`, `Dense` a `Sigmoid` rovnako ako triedu `MLP` pre samotnú neurónovú sieť.

Skript ďalej obsahuje niekoľko testových vstupov, na ktorom neskôr otestujeme vaše riešenia.

Na začiatku zavoláme potrebnú knižnicu `numpy` pre podporu výpočtov s maticami rôznych rozmerov. Následne nastavíme generovanie náhodných čísel, čo neskôr využijeme pre inicializáciu váh, aby naše pokusy boli opakovateľné:

In [ ]:
import numpy as np


np.random.seed(42)  # set random number generator for reproducability

## 2. Trieda `Layer`

Prvá trieda definovaná v skripte je tzv. dummy trieda, ktorá je veľmi podobná abstraktným triedam a reprezentuje všeobecnú funkcionalitu vrstvy neurónovej siete. Túto triedu reálne nikdy nevyužijeme, budú však od nej dediť všetky ostatné implementované triedy. Práve preto konštruktor tejto triedy je prázdny, implementovaný je iba prechodná časť, teda funkcia `forward`, ktorá vráti iba hodnoty na vstupe. Funkciu `backward` nebudeme implementovať, ak sa rozhodnite vaše zadania vypracovať na základe tohto riešenia, môžete tu pridať všeobecný spôsob trénovania vrstiev v neurónovej sieti.

In [ ]:
class Layer:
    """
    This is just a dummy class that is supposed to represent the general
    functionality of a neural network layer. Each layer can do two things:
     - forward pass - prediction
     - backward pass - training
    """

    def __init__(self):
        pass

    def forward(self, inp):
        # a dummy layer returns the input
        return inp

    def backward(self, inp, grad_outp):
        pass

## 3. Trieda `ReLU`

V ďalšom kroku implementujeme triedu ReLU, ktorá reprezentuje aktivačnú funkciu [ReLU](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7). Naše riešenie teda rozdeľuje výpočet váženej sumy od výpočtu výstupu aktivačnej funkcie - akokeby sme rozdelili jednu vrstvu na dve vrstvy: jedna pre výpočet sumy, jedna pre aktivačnú funkciu. Pri implementácii neurónovej siete tento rozdiel v reprezentácii skryjeme ako implementačný detail.

Aktivačná funkcia je veľmi jednoduchá funkcia, ktorá sa používa najmä v hlbokom učení, ale vzhľadom na jej jednoduchosť ju použijeme aj v tomto kroku, aby sme vedeli jednoduchšie otestovať naše riešenie. Vzorec ReLU je nasledovný:

$ReLU(x) = \left\{\begin{matrix}
x & ak x > 0\\ 
0 & naopak
\end{matrix}\right.$

Urobte analýzu triedy, navrhnite a implementujte riešenie - konštruktor a funkciu `forward`, pre ktorú vstupom bude výsledok z váženej sumy.

In [ ]:
class ReLU(Layer):
    def __init__(self):
        pass

    def forward(self, inp):
        return 0.0

    def backward(self, inp, grad_outp):
        pass

## 4. Trieda `Dense`

V tomto kroku implementujete triedu pre plne prepojenú vrstvu (po anglicky *fully-connected layer* alebo *dense layer*). Vstupom do tejto vrstvy sú vstupné dáta alebo výstupy predošlej (aktivačnej) vrstvy. Urobte analýzu triedy a navrhnite riešenie pre konštruktor a funkciu `forward`.

Konštruktor má nasledujúce parametre:
* `inp_units` - počet vstupov do každého neurónu, teda počet neurónov v predošlej vrstve
* `outp_units` - počet výstupu, teda počet neurónov v danej vrstve
* `learning_rate` - hodnota učiaceho parametra, ktorý hrá rolu pri trénovanie neurónky

Funkcia `forward` má jediný parameter:
* `inp` - vektor vstupných hodnôt do každého jedného neurónu v danej vrstve

Triedu môžete rozšíriť o rôzne ďalšie potrebné metódy.

In [ ]:
class Dense(Layer):
    def __init__(self, inp_units, outp_units, learning_rate=0.1):
        pass

    def forward(self, inp):
        return 0.0

    def backward(self, inp, grad_outp):
        pass

## 5. Trieda `MLP`

Ak máme implementovanú vrstvu pre výpočet váženej sumy a pre aktivačnú funkciu, môžeme z nich vytvoriť neurónovú sieť, teda viacvrstvový perceptrón. Trieda `MLP` je určený pre tento účel a definuje nasledujúce funkcie:

* `__init__` - konštruktor triedy, bez parametrov
* `add_layer` - pridá vrstvu do neurónovej siete; vašou úlohou je skryť pred používateľom vášho riešenia implementačné detaily (vrstva je reálne rozdelená do dvoch vrstiev), práve preto použijeme rozhranie, ktoré je veľmi bežné pre rôzne knižnice na vytvorenie neurónových sietí:
  * `neuron_count` - počet neurónov v danej vrstve
  * `inp_shape` - tvar vstupu pre danú vrstvu; defaultne je `None`, používateľ ho potrebuje zadefinovať iba pre prvú vrstvu, pre ďalšie vrstvy sa určí na základe predošlej vrstvy
  * `activation` - aktivačná funkcia použitá v danej vrstve; defaultne má hodnotu ReLU, môžete už pridať podporu pre sigmoidálnu funkciu (aj keď zatiaľ nie je implementovaná)
* `forward` - funkcia vypočíta výsledok doprednej fázy neurónovej siete pre vstup `X`; pre zjednodušenie ladenia programu vám odporúčame, aby funkcia nevrátila iba celkový výsledok (výstup z výstupnej vrstvy) ale výstup pre každú vrstvu (medzivýsledky postupne pridávajte do zoznamu `activations`)
* `predict` - funkcia vráti predikciu neurónovej siete pre vstup `X`; viacvrstvový perceptrón sa používa pre klasifikáciu, predikcia má byť index najvyššej hodnoty vo výstupe z výstupnej vrstvy
* `fit` - funkcia slúži na trénovanie neurónovej siete pre vstup `X` a očakávaný výstup `y`; zatiaľ ju nebudeme implementovať

Urobte analýzu triedy a následne ju implementujte na základe navrhnutého riešenia.

In [ ]:
class MLP():
    def __init__(self):
        pass

    def add_layer(self, neuron_count, inp_shape=None, activation='sigmoid'):
        pass

    def forward(self, X):
        activations = []

        return activations

    def predict(self, X):
        return 0.0

    def fit(self, X, y):
        pass

## 6. Testovanie riešenia

Ak ste úspešne implementovali vrstvy a sieť, môžete vaše riešenie vyskúšať na reálnom príklade. V metóde `main` máte definované dve vstupy s dĺžkou tri (pole `test`).

Do premennej `network` pridajte vrstvy a otestujte korektnosť riešenia zavolaním funkcie `predict`. Alternatívne, môžete vypísať aj výstup z funkcie `forward`, aby ste vedeli skontrolovať aj medzivýsledky. Odporúčame vypísať aj hodnoty váh v jednotlivých vrstvách, aby ste vedeli porovnať výstup neurónovej siete s očakávaným výstupom.

In [ ]:
test = [[300, 400, 500], [2, 0, 1]]
test = np.array(test)

network = MLP()

# TODO: add layers to the network

print(network.predict(test))

## 7. Vrstva `Sigmoid`

Ak vaše riešenie funguje správne, môžete ho rozšíriť triedou a teda aktivačnou funkciou `Sigmoid`. Implementácia bude veľmi podobná vrstve `ReLU`, iba použijete iný spôsob výpočtu výsledku:

$sigmoid(x) = \frac{1}{1 + e^{-x}}$

In [ ]:
class Sigmoid(Layer):
    def __init__(self):
        pass

    def forward(self, inp):
        return 0.0

    def backward(self, inp, grad_outp):
        pass

Následne môžete zadefinovať nový model so sigmoidálnou aktivačnou funkciou a otestovať jeho fungovanie. Sústreďte sa na rozdiely medzi dvomi aktivačnými funkciami:

In [ ]:
test = [[300, 400, 500], [2, 0, 1]]
test = np.array(test)

network = MLP()

# TODO: add layers to the network

print(network.predict(test))

Ukážkové riešenie cvičenia nájdete na [tejto adrese](sources/lab03/lab03-mlp-solution.py).